# Évaluation des recommandations

Pour évaluer nos systèmes de recommandations, nous utilisons les notes que les utilisateurs ont données aux séries comme jugement de pertinence (si un utilisateur a mis plus de 7, on met un jugement de pertinence à 1, sinon, il est à 0). 
Pour chaque utilisateur, on cache une partie des notes qu'il a données et on recommande parmi ces séries là (en prédisant les notes que l'utilisateur va leur donner). On calcule ensuite la nDCG sur la suite de séries ordonnées renvoyée.

## Création des matrices

In [1]:
from utils.collaborative import *
from utils.predictions_content import *
from utils.predictions_notes import *
from utils.recommandation import *
from utils.eval_reco import *

Import collaborative.py ok
Import predictions_notes ok
Import predictions_content ok
Import similarities ok
Import ndcg ok
Import recommandation ok
Import eval_reco.py ok


In [2]:
#path_d_user = "/Users/constancescherer/Desktop/pickles/d_user.p"
#path_sim = "/Users/constancescherer/Desktop/pickles/sim.p"
#path_most_sim = "/Users/constancescherer/Desktop/pickles/most_sim.p"
#path_d_pert_user = "/Users/constancescherer/Desktop/pickles/d_pert_user_k3.p"

path_d_user = "/Vrac/PLDAC_addic7ed/pickles/d_user.p"
path_sim = "/Vrac/PLDAC_addic7ed/pickles/sim.p"
path_most_sim = "/Vrac/PLDAC_addic7ed/pickles/most_sim.p"
path_d_pert_user = "/Vrac/PLDAC_addic7ed/pickles/d_pert_user_k3.p"
path_d_pop = "/Vrac/PLDAC_addic7ed/pickles/d_pop.p"


# dictionnaire d_users
# {username : {serie : rating}}
with open(path_d_user, 'rb') as pickle_file:
    d_user = pickle.load(pickle_file)

# matrice des similarités cosinus
with open(path_sim, 'rb') as pickle_file:
    sim = pickle.load(pickle_file)

# dictionnaire des séries les plus similaires
with open(path_most_sim, 'rb') as pickle_file:
    most_similar = pickle.load(pickle_file)
    
with open(path_d_pert_user, 'rb') as pickle_file:
    d_pert_user = pickle.load(pickle_file)
    
with open(path_d_pop, 'rb') as pickle_file:
    d_pop = pickle.load(pickle_file)


In [3]:
#path_ratings = "/Users/constancescherer/Desktop/ratings/ratings_imdb/users"
path_ratings = "/Vrac/PLDAC_addic7ed/ratings/ratings_imdb/users"

liste_series = get_liste_series(d_user)
data = get_data(d_user)
all_data, num_user, num_item = get_all_data(data)
train, train_mat, test = get_train_test(num_user, num_item, all_data, test_size=10)
mean, u_means, i_means,U_ksvd, I_ksvd =  get_Uksvd_Iksvd(train, train_mat, num_user, num_item)
d_username_id, d_itemname_id, Full = create_sparse_mat(data)


#path_series = "/Users/constancescherer/Desktop/addic7ed_final"
path_series = '/Vrac/PLDAC_addic7ed/addic7ed_final'

d_info, d_name = getDicts(path_series)
d_ind = reverse_dict(d_name)
d_titre_filename = get_d_titre_filename("titles/title-filename.txt")
d_filename_titre = reverse_dict(d_titre_filename)
d_id_username = reverse_dict(d_username_id)
d_id_serie = reverse_dict(d_itemname_id)

reversed_u_dic, reversed_i_dic = create_reversed_dic(d_username_id, d_itemname_id)

In [4]:
truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
truth_te = np.array([rating for uid,iid,rating in test])   

## Filtrage collaboratif

### kSVD

In [5]:
prediction_tr = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for (u,i),rating in train_mat.items()])
prediction_te = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for u,i,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))

print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Training Error:
MSE: 1.4274376744000552
MAE: 0.7237943947559603
Test Error:
MSE: 5.616544363803645
MAE: 1.7333823568196423


In [6]:
print("Évaluation kSVD : ")
print("nDCG en train : ", get_ndcg_moy_train(prediction_tr, train_mat, d_id_username, d_id_serie))
print("nDCG en test : ", get_ndcg_moy_test(prediction_te, test, d_id_username, d_id_serie))

Évaluation kSVD : 
nDCG en train :  0.9139697999774664
nDCG en test :  0.6819401953614532


L'évaluation des recommandations en test est bien évidemment moins bonne qu'en train mais elle reste assez bonne.

### NMF

In [7]:
prediction_tr, prediction_te = predictions_NMF(train_mat,test, 100, num_user, num_item)
print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))

print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Training Error:
MSE: 0.0
MAE: 0.0
Test Error:
MSE: 8.797942689199118
MAE: 2.2909625275532695


In [8]:
print("Évaluation NMF : ")
print("nDCG en train : ", get_ndcg_moy_train(prediction_tr, train_mat, d_id_username, d_id_serie))
print("nDCG en test : ", get_ndcg_moy_test(prediction_te, test, d_id_username, d_id_serie))

Évaluation NMF : 
nDCG en train :  0.9309231762427372
nDCG en test :  0.6810336068066001


Très similaire au kSVD.

## Content

In [6]:
prediction_tr = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, sim) for (u,i),rating in train_mat.items()])
prediction_te = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, sim) for u,i,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))

print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Training Error:
MSE: 3.6669966255748765
MAE: 1.3577911121996353
Test Error:
MSE: 3.663003102294065
MAE: 1.3543350477590008


In [7]:
print("Évaluation content : ")
print("nDCG en train : ", get_ndcg_moy_train(prediction_tr, train_mat, d_id_username, d_id_serie))
print("nDCG en test : ", get_ndcg_moy_test(prediction_te, test, d_id_username, d_id_serie))

Évaluation content : 
nDCG en train :  0.9011661655874822
nDCG en test :  0.6906627862813159


Un peu meilleur que le filtrage collaboratif en test.

L'évaluation réalisée donne de bons résulats, mais il est très difficile d'évaluer un système de recommandation. En effet, ici, nous avons recommandé des séries que l'utilisateur avait déjà noté, et non des séries qu'il n'a pas encore vu.  
Il faudrait pouvoir noter les recommandations sur des séries que les utilisateurs n'ont pas vu, ce qui est impossible (pour nous, il nous faudrait des feedbacks des utilisateurs).

## Tri par popularité

Nous avons remarqué un problème dans nos recommandations : on renvoit souvent les mêmes (qui sont en plus des séries assez peu connues - on pense que ce problème vient de leur biais très élevé, car le peu de gens les ayant notées les ont très bien notées).  
Pour régler ce problème, nous recourrons à l'heuristique suivante lorsque les notes max prédites pour un utilisateur sont toutes les mêmes : on recommande les séries les plus vues / les mieux notées.

### kSVD

In [5]:
prediction_tr = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for (u,i),rating in train_mat.items()])
prediction_te = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for u,i,rating in test])

print("Évaluation kSVD : ")
print("nDCG en train : ", get_ndcg_moy_train_pop(prediction_tr, train_mat, d_id_username, d_id_serie, d_pop))
print("nDCG en test : ", get_ndcg_moy_test_pop(prediction_te, test, d_id_username, d_id_serie, d_pop))

Évaluation kSVD : 
nDCG en train :  0.8188542845592465
nDCG en test :  0.6879465662783582


### NMF

In [6]:
prediction_tr, prediction_te = predictions_NMF(train_mat,test, 100, num_user, num_item)

print("Évaluation NMF : ")
print("nDCG en train : ", get_ndcg_moy_train_pop(prediction_tr, train_mat, d_id_username, d_id_serie, d_pop))
print("nDCG en test : ", get_ndcg_moy_test_pop(prediction_te, test, d_id_username, d_id_serie, d_pop))

Évaluation NMF : 
nDCG en train :  0.8186379987801757
nDCG en test :  0.688055286613937


### Content

In [7]:
prediction_tr = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, sim) for (u,i),rating in train_mat.items()])
prediction_te = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, sim) for u,i,rating in test])

print("Évaluation content : ")
print("nDCG en train : ", get_ndcg_moy_train_pop(prediction_tr, train_mat, d_id_username, d_id_serie, d_pop))
print("nDCG en test : ", get_ndcg_moy_test_pop(prediction_te, test, d_id_username, d_id_serie, d_pop))

Évaluation content : 
nDCG en train :  0.8190327413361507
nDCG en test :  0.6879568289210404


Avec le tri par popularité, les recommandations sont améliorées en test pour le kSVD et la NMF.